# Indicadores por comuna

## Importación de datos y librerias

In [1]:
# Importa librerias necesarias para el análisis

import pandas as pd
import numpy as np
import plotly.express as px

In [4]:
# Importa datos de población regional y crea DF


df_poblacion = pd.read_csv('data/aysen_population.csv') #Archivo ubicado dentro de la carpeta data

df_poblacion.head(2)

,Unnamed: 0,orden,nombre_region,codigo_region,nombre_provincia,codigo_provincia,nombre_comuna,codigo_comuna,edad_hombres,edad_mujeres,total
0,0,7172,Región de Aysén del General Carlos Ibáñez del ...,11,Coyhaique,111,Coyhaique,11101,28763,29055,57818
1,1,7194,Región de Aysén del General Carlos Ibáñez del ...,11,Coyhaique,111,Lago Verde,11102,494,358,852


In [5]:
# Importa datos de comunas corregidas por establecimiento de salud

df_establecimiento = pd.read_excel('data/establecimientos_salud_dict.xlsx') #Archivo ubicado dentro de la carpeta data


df_establecimiento.head()

,establecimiento_salud,comuna_seremi_xi,provincia_seremi_xi
0,Actividades gestionadas por la Dirección del S...,Coyhaique,Coyhaique
1,Centro Clínico Militar Coyhaique,Coyhaique,Coyhaique
2,Centro Comunitario de Salud Familiar Villa Ma...,Aysen,Aysen
3,Centro Comunitario de Salud Familiar Alejandro...,Coyhaique,Coyhaique
4,Centro Comunitario de Salud Familiar Puerto Ch...,Aysen,Aysen


In [6]:
# Importa datos desde la base de datos recivida via FTP
## PRIMERO: Guardar archivo dentro de la carpeta data
## SEGUNDO: No cambiar nombre ni formato del archivo recibido
## TERCERO: RECORDAR CAMBIAR FECHA EN EL ARCHIVO A IMPORTAR A CONTINUACION, i.e: 20210829 para este domingo 29 de Agosto 2021

df =  pd.read_csv('data/20210909_Aisen_notificaciones_0600.csv', 
                   sep='~', # CSV con separador ''~''
                   low_memory=False
                  )

## Une datos con diccionario comunal

In [7]:
df_establecimiento.sample(3)

,establecimiento_salud,comuna_seremi_xi,provincia_seremi_xi
24,Posta de Salud Rural Bahía Murta,Rio Ibañez,General Carrera
23,Policlínico del Trabajador Coyhaique,Coyhaique,Coyhaique
31,Posta de Salud Rural La Tapera,Lago Verde,Coyhaique


In [8]:
dff = df.merge(df_establecimiento, how='left', on='establecimiento_salud')

## Integra datos poblacionales

Se realizará merge del dataset recientemente generado con el diccionario de estimación poblacional importado

In [9]:
df_merged = dff.merge(df_poblacion.drop(columns= ['Unnamed: 0','orden','nombre_region', 'codigo_region', 'nombre_provincia','codigo_provincia']), left_on='comuna_seremi_xi', right_on='nombre_comuna', how='left' )

## Limpieza de datos

Deben seleccionarse las variables de importancia y filtrarse observaciones de interes

In [10]:
# Filtra datos obtenidos solo para pacientes con residencia en la Región de Aysén

data_aysen = df_merged[df_merged['region_residencia'] == 'Región de Aysén del General Carlos Ibáñez del Campo']

data_aysen.head(1)

,id_formulario_eno,numero_folio,id_enfermedad_eno,enfermedad_notificada,semana_epidemiologica,fecha_notificacion,etapa_clinica,establecimiento_salud,region,seremi,...,tac_torax_covid,condicion,region_separacion,comuna_seremi_xi,provincia_seremi_xi,nombre_comuna,codigo_comuna,edad_hombres,edad_mujeres,total
0,100166,50166,50,Coronavirus (COVID-19),11,2020-03-12,DESCARTADA,Hospital Regional (Coihaique),Región de Aysén del General Carlos Ibáñez del ...,SEREMI De Aisén del General Carlos Ibañez del ...,...,NaN,NaN,Región de Aysén del General Carlos Ibáñez del ...,Coyhaique,Coyhaique,Coyhaique,11101.0,28763.0,29055.0,57818.0


## Adicionalmente datos deben ser filtrados para otras variables: 

|variable|tipo|valor a mantener|valor a eliminar|
|---|---|---|---|
|`seremi`|Text|SEREMI De Aisén del General Carlos Ibañez del Campo|otros|
|`estado_caso`|Text|Validada e Inconcluso|No validada|
|`vigente_no_eliminado`|Boolean|True|False|
|`enfermedad_notificada`|Text|Solo Coronavirus (COVID-19)||



In [11]:
# Formato del 20210831 con cambios con vigente_no_eliminado True|False
## Si el código anterior es decomentado (eliminando las comillas), este debe ser comentado es decir
## transformado en código que no correrá, mediante la adición de tres comillas al inicio y al final del código

data_aysen_filtered = data_aysen[(data_aysen['estado_caso']!='No validada')&(
                          data_aysen['vigente_no_eliminado']==True)&(
                          data_aysen['seremi']=='SEREMI De Aisén del General Carlos Ibañez del Campo')
                       ].rename(columns={'edad_hombres':'pop_hombres',
                                         'edad_mujeres':'pop_mujeres',
                                         'total':'pop_total'})


In [13]:
# Transforma Fechas a formato DateTime 

data_aysen_filtered['fecha_notificacion'] = pd.to_datetime(data_aysen_filtered['fecha_notificacion'], errors = 'coerce') 

data_aysen_filtered['fecha_toma_muestra_1'] = pd.to_datetime(data_aysen_filtered['fecha_toma_muestra_1'], errors = 'coerce')

data_aysen_filtered['fecha_toma_muestra_2'] = pd.to_datetime(data_aysen_filtered['fecha_toma_muestra_2'], errors = 'coerce')

data_aysen_filtered['fecha_toma_muestra_3'] = pd.to_datetime(data_aysen_filtered['fecha_toma_muestra_3'], errors = 'coerce')

data_aysen_filtered['fecha_resultado_pcr_1'] = pd.to_datetime(data_aysen_filtered['fecha_resultado_pcr_1'], errors = 'coerce')

data_aysen_filtered['fecha_resultado_pcr_2'] = pd.to_datetime(data_aysen_filtered['fecha_resultado_pcr_2'], errors = 'coerce')

data_aysen_filtered['fecha_resultado_pcr_3'] = pd.to_datetime(data_aysen_filtered['fecha_resultado_pcr_3'], errors = 'coerce')

## Cálculo de semana epidemiológica

In [14]:
# Agrega 53 semanas a las semanas de 2021 y crea semana_epidemiologica_21

data_aysen_clean1 = data_aysen_filtered[data_aysen_filtered['fecha_notificacion']<=pd.to_datetime('2021-01-02')]
data_aysen_clean2 = data_aysen_filtered[data_aysen_filtered['fecha_notificacion']>=pd.to_datetime('2021-01-03')]


data_aysen_clean1['semana_epidemiologica_21'] = data_aysen_clean1['semana_epidemiologica']
data_aysen_clean2['semana_epidemiologica_21'] = data_aysen_clean2['semana_epidemiologica']+53

data_aysen_clean_21 = data_aysen_clean1.append(data_aysen_clean2, ignore_index=True)


<ipython-input-14-9c1e743327bc>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aysen_clean1['semana_epidemiologica_21'] = data_aysen_clean1['semana_epidemiologica']
<ipython-input-14-9c1e743327bc>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aysen_clean2['semana_epidemiologica_21'] = data_aysen_clean2['semana_epidemiologica']+53


## Gráficos de interes

In [16]:
# Muestra datos por semana epidemiologica 
# Muestra datos de tests realizados en semana_x

data_aysen_clean_21.groupby(['semana_epidemiologica_21', 'tipo_caso_busqueda', 'comuna_seremi_xi'])[['numero_folio']].count().rename(columns={'numero_folio': 'numero_tests'}).reset_index().pivot(
                            index=['semana_epidemiologica_21', 'tipo_caso_busqueda'],
                            columns='comuna_seremi_xi', 
                            values='numero_tests'
).fillna(0).reset_index().melt(id_vars=['semana_epidemiologica_21', 'tipo_caso_busqueda'], 
        var_name='comuna_seremi_xi', 
        value_name='casos')

,semana_epidemiologica_21,tipo_caso_busqueda,comuna_seremi_xi,casos
0,12,Atención médica,Aysen,3.0
1,12,Búsqueda activa,Aysen,0.0
2,13,Atención médica,Aysen,4.0
3,13,Búsqueda activa,Aysen,0.0
4,14,Atención médica,Aysen,7.0
...,...,...,...,...
1579,87,Búsqueda activa,Tortel,1.0
1580,88,Atención médica,Tortel,2.0
1581,88,Búsqueda activa,Tortel,3.0
1582,89,Atención médica,Tortel,0.0


In [18]:
# Crea gráfico de BAC regional por semana epidemiologica

px.bar(data_aysen_clean_21.groupby(['semana_epidemiologica_21', 'tipo_caso_busqueda', 'comuna_seremi_xi'])[['numero_folio']].count().rename(columns={'numero_folio': 'numero_tests'}).reset_index().pivot(
                            index=['semana_epidemiologica_21', 'tipo_caso_busqueda'],
                            columns='comuna_seremi_xi', 
                            values='numero_tests'
                        ).fillna(0).reset_index().melt(id_vars=['semana_epidemiologica_21', 'tipo_caso_busqueda'], 
        var_name='comuna_seremi_xi', 
        value_name='casos'),
        x= 'semana_epidemiologica_21',
        y= 'casos',
        color= 'tipo_caso_busqueda',
        color_discrete_map={
        'Búsqueda activa': '#DD4477',
        'Atención médica': '#0099C6'
        },
        facet_col='comuna_seremi_xi',
        facet_col_wrap=4,
       # color_continuous_scale='Viridis',
        labels={'tipo_caso_busqueda':'Caso de búsqueda'},
       ).update_layout(title= 'Cantidad de test PCR realizados en la Región de Aysén',
                       yaxis_title='Total de test PCR realizados',
                       xaxis_title='Semana epidemiológica',
                       template='simple_white',
                     #  xaxis=dict(tickmode = 'array',
                      #            tickvals = list(range(12,88,3)),
                       #           ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                        #         ),
                    ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                               #).add_hline(y=2000, line_width=3, line_dash='dash', line_color='#0099C6'
                                          ).add_vrect(x0='53', x1='50', 
                                           annotation_text='2020', 
                                           annotation_position='top right',
                                           annotation=dict(font_size=14, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                          ).add_vrect(x0='84', x1='86', 
                                           annotation_text='2021', 
                                           annotation_position='top right',
                                           annotation=dict(font_size=14, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                                     ).update_xaxes(title_text='', row=1, col=1
                                                     ).update_xaxes(title_text='', row=1, col=2
                                                     ).update_xaxes(title_text='Semana epidemiológica', row=1, col=3
                                                     ).update_xaxes(title_text='', row=1, col=4
                                                     ).update_xaxes(title_text='', row=1, col=5
                                                     ).update_yaxes(title_text='', row=1, col=1, 
                                                     ).update_yaxes(title_text='', row=2, col=1,
                                                     ).update_yaxes(title_text='', row=3, col=1, 
                                                     ).update_yaxes(title_text='', row=4, col=1 
                                                     ).for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1])).add_annotation(x=-0.08,y=0.4,
                   text='Total de test PCR realizados', textangle=-90,
                    xref="paper", yref="paper",
                     font_size=14, 
                     font_family="'Open Sans', verdana, arial, sans-serif",
                    # opacity=0,                                                                                       
) 

In [20]:
data_aysen_clean_21.head(1)

,id_formulario_eno,numero_folio,id_enfermedad_eno,enfermedad_notificada,semana_epidemiologica,fecha_notificacion,etapa_clinica,establecimiento_salud,region,seremi,...,condicion,region_separacion,comuna_seremi_xi,provincia_seremi_xi,nombre_comuna,codigo_comuna,pop_hombres,pop_mujeres,pop_total,semana_epidemiologica_21
0,100166,50166,50,Coronavirus (COVID-19),11,2020-03-12,DESCARTADA,Hospital Regional (Coihaique),Región de Aysén del General Carlos Ibáñez del ...,SEREMI De Aisén del General Carlos Ibañez del ...,...,NaN,Región de Aysén del General Carlos Ibáñez del ...,Coyhaique,Coyhaique,Coyhaique,11101.0,28763.0,29055.0,57818.0,11


In [54]:
df_tasa_caso_busqueda = data_aysen_clean_21[data_aysen_clean_21['comuna_seremi_xi']!='Ejercicio Libre de la Profesión'].groupby(['semana_epidemiologica_21', 'tipo_caso_busqueda', 'comuna_seremi_xi'])[['numero_folio']].count(
                                ).reset_index().merge(data_aysen_clean_21[['comuna_seremi_xi','pop_total']].drop_duplicates(),
                                        on='comuna_seremi_xi',
                                        how='left'
                                       )

In [55]:
df_tasa_caso_busqueda['tasa'] = df_tasa_caso_busquea['numero_folio'] / df_tasa_caso_busquea['pop_total']*100000

In [64]:
df_tasa_caso_busqueda_ratio = df_tasa_caso_busqueda.pivot(
                                    index=['semana_epidemiologica_21','comuna_seremi_xi'],
                                    columns='tipo_caso_busqueda', 
                                    values='tasa'
                                ).fillna(0)

In [68]:
df_tasa_caso_busqueda_ratio['ratio'] = (df_tasa_caso_busqueda_ratio['Búsqueda activa']/(df_tasa_caso_busqueda_ratio['Búsqueda activa']+df_tasa_caso_busqueda_ratio['Atención médica']))*100

In [79]:
df_tasa_caso_busqueda_ratio.rolling(7).mean().fillna(0)

tipo_caso_busqueda                         Atención médica  Búsqueda activa  \
semana_epidemiologica_21 comuna_seremi_xi                                     
12                       Aysen                    0.000000         0.000000   
                         Cisnes                   0.000000         0.000000   
                         Coyhaique                0.000000         0.000000   
13                       Aysen                    0.000000         0.000000   
                         Coyhaique                0.000000         0.000000   
...                                                    ...              ...   
89                       Cochrane              1073.290176       956.390487   
                         Coyhaique             1053.089531       804.041604   
                         Guaitecas             1088.726002       738.215486   
                         Rio Ibañez            1139.632819       468.590987   
                         Tortel                 783.513293       468.220464   

tipo_caso_busqueda                             ratio  
semana_epidemiologica_21 comuna_seremi_xi             
12                       Aysen              0.000000  
                         Cisnes             0.000000  
                         Coyhaique          0.000000  
13                       Aysen              0.000000  
                         Coyhaique          0.000000  
...                                              ...  
89                       Cochrane          55.770137  
                         Coyhaique         55.611497  
                         Guaitecas         52.414707  
                         Rio Ibañez        41.150970  
                         Tortel            53.536677  

[570 rows x 3 columns]

In [96]:
# Crea gráfico de BAC regional por semana epidemiologica

px.scatter(df_tasa_caso_busqueda_ratio.rolling(14).mean().reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'ratio',
        color= 'ratio',
        #color_discrete_map={
        #'Búsqueda activa': '#DD4477',
        #'Atención médica': '#0099C6'
        #},
        facet_col='comuna_seremi_xi',
        facet_col_wrap=2,
        trendline='lowess',
        #groupnorm='percent',
        color_continuous_scale=px.colors.sequential.Viridis_r,
        labels={'tipo_caso_busqueda':'Caso de búsqueda'},
       ).update_layout(title= 'Ind1: Test PCR realizados en la Región de Aysén por Búsqueda Activa',
                       yaxis_title='Total de test PCR realizados',
                       xaxis_title='Semana epidemiológica',
                       template='simple_white',
                     #  xaxis=dict(tickmode = 'array',
                      #            tickvals = list(range(12,88,3)),
                       #           ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                        #         ),
                    ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                               #).add_hline(y=2000, line_width=3, line_dash='dash', line_color='#0099C6'
                                          ).add_vrect(x0='53', x1='50', 
                                           annotation_text='2020', 
                                           annotation_position='bottom right',
                                           annotation=dict(font_size=14, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                          ).add_vrect(x0='84', x1='86', 
                                           annotation_text='2021', 
                                           annotation_position='bottom right',
                                           annotation=dict(font_size=14, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                                     ).update_xaxes(title_text='', row=1, col=1
                                                     ).update_xaxes(title_text='', row=1, col=2
                                                     ).update_xaxes(title_text='Semana epidemiológica', row=1, col=3
                                                     ).update_xaxes(title_text='', row=1, col=4
                                                     ).update_xaxes(title_text='', row=1, col=5
                                                     ).update_yaxes(title_text='', row=1, col=1, 
                                                     ).update_yaxes(title_text='', row=2, col=1,
                                                     ).update_yaxes(title_text='', row=3, col=1, 
                                                     ).update_yaxes(title_text='', row=4, col=1,
                                                     ).update_yaxes(title_text='', row=5, col=1 
                                                     ).for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1])).add_annotation(x=-0.08,y=0.4,
                   text='Total de test PCR realizados', textangle=-90,
                    xref="paper", yref="paper",
                     font_size=14, 
                     font_family="'Open Sans', verdana, arial, sans-serif",
                    # opacity=0,                                                                                       
) 